In [10]:
import os
import json
import numpy as np
import cv2
import shutil

from tqdm import tqdm

In [ ]:

class_info = [
    {'label':'pelagic_sediments', 'type':'polygon', 'color':1},
    {'label':'basalt','type':'polygon','color':2},
    {'label':'basalt_breccia','type':'polygon','color':3},
    {'label':'polymetallic_sulfide','type':'polygon','color':4}
]

In [ ]:
def labelme2mask_single_img(img_path, labelme_json_path):
    '''
 
    '''
    
    img_bgr = cv2.imread(img_path)
    img_mask = np.zeros(img_bgr.shape[:2]) 
    
    with open(labelme_json_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    for one_class in class_info: 
        for each in labelme['shapes']: 
            if each['label'] == one_class['label']:
                if one_class['type'] == 'polygon': 

                    
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    
                    img_mask = cv2.fillPoly(img_mask, points, color=one_class['color'])

                elif one_class['type'] == 'line' or one_class['type'] == 'linestrip': 

                    
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    
                    img_mask = cv2.polylines(img_mask, points, isClosed=False, color=one_class['color'], thickness=one_class['thickness']) 

                elif one_class['type'] == 'circle': 

                    points = np.array(each['points'], dtype=np.int32)

                    center_x, center_y = points[0][0], points[0][1] 

                    edge_x, edge_y = points[1][0], points[1][1]     

                    radius = np.linalg.norm(np.array([center_x, center_y] - np.array([edge_x, edge_y]))).astype('uint32') 

                    img_mask = cv2.circle(img_mask, (center_x, center_y), radius, one_class['color'], one_class['thickness'])

                else:
                    print('未知标注类型', one_class['type'])
                    
    return img_mask

In [ ]:
os.chdir(Dataset_Path)
os.mkdir('masks')
os.chdir('images')

In [ ]:
for img_path in tqdm(os.listdir()):
    
    try:
    
        labelme_json_path = os.path.join('../', 'labelme_jsons', '.'.join(img_path.split('.')[:-1])+'.json')

        img_mask = labelme2mask_single_img(img_path, labelme_json_path)

        mask_path = img_path.split('.')[0] + '.png'

        cv2.imwrite(os.path.join('../','masks',mask_path), img_mask)
    
    except Exception as E:
        print(img_path, '转换失败', E)
        

In [ ]:
os.chdir('../')
shutil.move('images', 'img_dir')
shutil.move('masks', 'ann_dir')
!rm -rf labelme_jsons
os.chdir('../')

In [ ]:
import seedir as sd
sd.seedir('', style='emoji', depthlimit=1)